In [2]:
import sqlite3
conn = sqlite3.connect('nominations.db')

schema = conn.execute('PRAGMA TABLE_INFO(nominations);').fetchall()
firstt = conn.execute('SELECT * FROM nominations LIMIT 10;').fetchall()

for each in schema:
    print(each)
    
for each in firstt:
    print(each)

(0, 'Year', 'INTEGER', 0, None, 0)
(1, 'Category', 'TEXT', 0, None, 0)
(2, 'Nominee', 'TEXT', 0, None, 0)
(3, 'Won', 'INTEGER', 0, None, 0)
(4, 'Movie', 'TEXT', 0, None, 0)
(5, 'Character', 'TEXT', 0, None, 0)
(2010, 'Actor -- Leading Role', 'Javier Bardem', 0, 'Biutiful', 'Uxbal')
(2010, 'Actor -- Leading Role', 'Jeff Bridges', 0, 'True Grit', 'Rooster Cogburn')
(2010, 'Actor -- Leading Role', 'Jesse Eisenberg', 0, 'The Social Network', 'Mark Zuckerberg')
(2010, 'Actor -- Leading Role', 'Colin Firth', 1, "The King's Speech", 'King George VI')
(2010, 'Actor -- Leading Role', 'James Franco', 0, '127 Hours', 'Aron Ralston')
(2010, 'Actor -- Supporting Role', 'Christian Bale', 1, 'The Fighter', 'Dicky Eklund')
(2010, 'Actor -- Supporting Role', 'John Hawkes', 0, "Winter's Bone", 'Teardrop')
(2010, 'Actor -- Supporting Role', 'Jeremy Renner', 0, 'The Town', 'James Coughlin')
(2010, 'Actor -- Supporting Role', 'Mark Ruffalo', 0, 'The Kids Are All Right', 'Paul')
(2010, 'Actor -- Supporting 

In [10]:
# create table
conn.execute('CREATE TABLE ceremonies (id integer primary key, year integer, host text);')

# prepare list of hosts
years_hosts = [(2010, "Steve Martin"),
               (2009, "Hugh Jackman"),
               (2008, "Jon Stewart"),
               (2007, "Ellen DeGeneres"),
               (2006, "Jon Stewart"),
               (2005, "Chris Rock"),
               (2004, "Billy Crystal"),
               (2003, "Steve Martin"),
               (2002, "Whoopi Goldberg"),
               (2001, "Steve Martin"),
               (2000, "Billy Crystal"),
            ]

# prepare and execute insert
insert_query = "INSERT INTO ceremonies (Year, Host) VALUES (?,?);"
conn.executemany(insert_query, years_hosts)

# verify
print(conn.execute('SELECT * FROM ceremonies LIMIT 10;').fetchall())
print(conn.execute('PRAGMA TABLE_INFO(ceremonies);').fetchall())

[(1, 2010, 'Steve Martin'), (2, 2009, 'Hugh Jackman'), (3, 2008, 'Jon Stewart'), (4, 2007, 'Ellen DeGeneres'), (5, 2006, 'Jon Stewart'), (6, 2005, 'Chris Rock'), (7, 2004, 'Billy Crystal'), (8, 2003, 'Steve Martin'), (9, 2002, 'Whoopi Goldberg'), (10, 2001, 'Steve Martin')]
[(0, 'id', 'integer', 0, None, 1), (1, 'year', 'integer', 0, None, 0), (2, 'host', 'text', 0, None, 0)]


In [9]:
conn.execute("DROP TABLE ceremonies").fetchall()

[]

In [11]:
conn.execute("PRAGMA foreign_keys = ON;").fetchall()

[]

In [13]:
create_nominations_two = '''
CREATE TABLE nominations_two 
    (id integer primary key, 
    category text, 
    nominee text, 
    movie text, 
    character text, 
    won text,
    ceremony_id integer,
FOREIGN KEY(ceremony_id) REFERENCES ceremonies(id));
'''

nom_query = '''
SELECT
    ceremonies.id as ceremony_id,
    nominations.category as category, 
    nominations.nominee as nominee,
    nominations.movie as movie, 
    nominations.character as character,
    nominations.won as won
FROM nominations
INNER JOIN ceremonies 
ON nominations.year == ceremonies.year;
'''
joined_nominations = conn.execute(nom_query).fetchall()

conn.execute(create_nominations_two)

insert_nominations_two = '''
INSERT INTO nominations_two (ceremony_id, category, nominee, movie, character, won) 
VALUES (?,?,?,?,?,?);
'''

conn.executemany(insert_nominations_two, joined_nominations)
print(conn.execute("SELECT * FROM nominations_two LIMIT 5;").fetchall())

[(1, 'Actor -- Leading Role', 'Javier Bardem', 'Biutiful', 'Uxbal', '0', 1), (2, 'Actor -- Leading Role', 'Jeff Bridges', 'True Grit', 'Rooster Cogburn', '0', 1), (3, 'Actor -- Leading Role', 'Jesse Eisenberg', 'The Social Network', 'Mark Zuckerberg', '0', 1), (4, 'Actor -- Leading Role', 'Colin Firth', "The King's Speech", 'King George VI', '1', 1), (5, 'Actor -- Leading Role', 'James Franco', '127 Hours', 'Aron Ralston', '0', 1)]


In [16]:
conn.execute("DROP TABLE nominations;")
conn.execute("ALTER TABLE nominations_two RENAME TO nominations;")

OperationalError: no such table: nominations

In [ ]:
create_movies_actors = '''
CREATE TABLE movies_actors
(id INTEGER PRIMARY KEY,
movie_id INTEGER references movies(id),
actor_id INTEGER references actors(id));
'''
conn.execute("CREATE TABLE movies (id integer primary key, movie text);")
conn.execute("CREATE TABLE actors (id integer primary key,actor text);")
conn.execute(create_movies_actors)